In [6]:
import tabula

# Specify the path to the PDF file
pdf_path = './AB1_CUT.pdf'
outputfilename = './Final_AST_AB1.csv'
textfilename = './AB1.txt'

In [7]:
# Read the table from the PDF file
dfs = tabula.read_pdf(pdf_path,  multiple_tables=True, pages='all')
# Specify the path to the CSV file

In [8]:
dfs[2]

,Antimicrobial,MIC,Interpretation,Antimicrobial.1,MIC.1,Interpretation.1
0,+Amoxicillin,NaN,R,Ceftolozane/Tazobactam,NaN,NaN
1,+Ampicillin,NaN,R,Cefepime,>= 32,R
2,+Amoxicillin/Clavulanic Acid,NaN,R,+Aztreonam,NaN,R
3,Ampicillin/Sulbactam,16,I,Imipenem,>= 16,R
4,+Piperacillin,NaN,R,Meropenem,>= 16,R
5,Piperacillin/Tazobactam,>= 128,R,Amikacin,>= 64,R
6,+Cefaclor,NaN,R,Gentamicin,NaN,NaN
7,+Cefuroxime,NaN,R,Ciprofloxacin,>= 4,R
8,+Cefixime,NaN,R,+Levofloxacin,NaN,R
9,+Cefpodoxime,NaN,R,+Norfloxacin,NaN,R


# Checking repeated patients

In [9]:
patient_names_list = []
patient_names_list_total =[]
name_to_be_add = []
ii = 0
with open(textfilename,'r') as f:
    lines = f.readlines()
    for line in lines:
            
        ls = line.split()
        
        if len(ls) <=2:
             continue
        if ls[0] == 'Patient' and ls[1] == 'Name:':
            # print(ls)
            
            # print(ls)
            # ii +=1 
            flag = True
            for ind in range(2,len(ls)):
                if ls[ind] == 'Patient':
                    name_to_be_add = ''.join(ls[2:ind])
                    break
                    
        if ls[0] == 'Susceptibility':
            patient_names_list_total.append(name_to_be_add)
            if name_to_be_add not in patient_names_list:
                patient_names_list.append(name_to_be_add)
                ii += 1
        
print(patient_names_list,len(patient_names_list))
print(patient_names_list_total,len(patient_names_list_total))
print(ii)
print('Test: repeated patient')
for x in patient_names_list:
    if patient_names_list_total.count(x) > 1:
        print(x)

['R858', 'D90', 'N1092'] 3
['R858', 'D90', 'N1092'] 3
3
Test: repeated patient


# new_method checking for duplicates

In [10]:
patient_id = []
ii =1 
with open('./AB1.txt','r') as f:
    lines = f.readlines()
    for line in lines:
        ii=ii+1
        # if ii<10:
        #     print(line)
        
        ls = line.split()
        if len(ls)>=2 and ls[0] == 'Patient' and ls[1]=='Name:':
            # print(ls)
            #print(ls)
            patient_id.append(ls[2])
            if ls[2] == 'R':
                print(ls)
patient_id_unique = []
patient_id_unique.append( patient_id[0])
for x in patient_id:
    if x != patient_id_unique[-1]:
        if x in patient_id_unique:
            print('repeat',x)
        patient_id_unique.append(x)
# Removing leading hyphen from each sample name in the list
patient_id_unique = [name.lstrip('-') for name in patient_id_unique]        
print(patient_id_unique)
print(len(patient_id_unique))

['R858', 'D90', 'N1092']
3


In [11]:
# Iterate over the tables and save each one as a CSV file
name_id = 0
total_tab = 0
out_name = []

for i, df in enumerate(dfs):
    df_cl = df.columns
    if 'Antimicrobial' in df_cl and 'MIC' in df_cl:
        total_tab += 1
        for ii in range(len(df)):
            for x in df.columns:
                a = str(df.loc[ii,x]).replace('\r','')
                df.loc[ii,x] = a
                if df.loc[ii,x] == 'Amoxicillin/Clavulanic A':
                    df.loc[ii,x] = 'Amoxicillin/Clavulanic Acid'
                elif df.loc[ii,x] =='Trimethoprim/':
                    df.loc[ii,x] = 'Trimethoprim/Sulfamethoxazole'
                elif df.loc[ii,x] == 'Sulfamethoxazole' :
                    df.loc[ii,x] = None
                elif df.loc[ii,x] == 'Gentamicin High Level' :
                    df.loc[ii,x] = 'Gentamicin High Level(synergy)' 
                elif df.loc[ii,x] == '(synergy)':
                    df.loc[ii,x] = None
                elif df.loc[ii,x] == 'Streptomycin High Level' :
                    df.loc[ii,x] = 'Streptomycin High Level(synergy)' 
                elif df.loc[ii,x] == 'Inducible Clindamycin':
                    df.loc[ii,x] = 'Inducible Clindamycin Resistance'
                elif df.loc[ii,x] == 'Resistance':
                    df.loc[ii,x] = None
                elif df.loc[ii,x]=='Inducible ClindamycinResistance':
                    df.loc[ii,x] = 'Inducible Clindamycin Resistance'
                
                
        if patient_names_list[name_id] not in out_name:
            out_name.append(patient_names_list[name_id])
            df.to_csv('./'+patient_names_list[name_id]+'.csv')
        
        name_id += 1
        # else: 
            # print(patient_names_list[name_id])
            
# print(name_id, len(patient_names_list))
# print(out_name, len(out_name))
# print(total_tab)

In [13]:
dfs[2]

,Antimicrobial,MIC,Interpretation,Antimicrobial.1,MIC.1,Interpretation.1
0,+Amoxicillin,nan,R,Ceftolozane/Tazobactam,nan,nan
1,+Ampicillin,nan,R,Cefepime,>= 32,R
2,+Amoxicillin/Clavulanic Acid,nan,R,+Aztreonam,nan,R
3,Ampicillin/Sulbactam,16,I,Imipenem,>= 16,R
4,+Piperacillin,nan,R,Meropenem,>= 16,R
5,Piperacillin/Tazobactam,>= 128,R,Amikacin,>= 64,R
6,+Cefaclor,nan,R,Gentamicin,nan,nan
7,+Cefuroxime,nan,R,Ciprofloxacin,>= 4,R
8,+Cefixime,nan,R,+Levofloxacin,nan,R
9,+Cefpodoxime,nan,R,+Norfloxacin,nan,R


In [14]:
def clear_str(s):
    string = s.replace("/ ", "/").replace("+", "").rstrip().lstrip()
    # string = string.replace('-','/').replace("=","")
    # if not '/' in string and ' ' in string:
    #     ind = string.index(' ')
    #     string = string[0:ind]+'/'+ string[ind + 1:]
        
    if '/' in string:
        index = string.index('/') + 1
        modified_string = string[0].upper()+string[1:index].lower()+ string[index].upper() + string[index + 1:].lower()
        return modified_string
    else:
        return string

In [15]:
import pandas as pd
mdr_folder_path = './'
new_column_dict = {}
for filename in out_name:
    file = pd.read_csv(mdr_folder_path+filename+'.csv')
    cl = []
    for x in file.columns:
        if 'Antimicrobial' in x:
                cl.append(x)
    for x in cl:
        antimicrobial_list = file.loc[:,x]
        # print(antimicrobial_list)
        for rl in antimicrobial_list:
            if str(rl) == "nan":
                continue
            
            rl0 = clear_str(str(rl))
            if rl0=='Ertapenem':
                print(rl)
                print(filename)
            
            
            # if rl0 ==   "Rtrimethoprim/Sulfamethoxazole": #"Trimethoprim/Sulfamethoxazole":
            #     print(filename)
            
            _ = new_column_dict.setdefault('MIC_'+str(rl0), 'MIC_'+str(rl0))
            _ = new_column_dict.setdefault(str(rl0), rl0)

In [16]:
new_column_list = []
new_column_name = []

show_list = []
for key,value in enumerate(new_column_dict):
    new_column_name.append(new_column_dict[value])
    new_column_list.append(value)
    if not new_column_dict[value].startswith('MIC'):
        show_list.append(new_column_dict[value])
        # print(new_column_dict[value])
        
for x in sorted(show_list):  
    print(x)
print(len(show_list))

Amikacin
Amoxicillin
Amoxicillin/Clavulanic acid
Ampicillin
Ampicillin/Sulbactam
Aztreonam
Cefaclor
Cefepime
Cefixime
Cefotaxime
Cefpodoxime
Ceftazidime
Ceftazidime/Avibactam
Ceftizoxime
Ceftolozane/Tazobactam
Ceftriaxone
Cefuroxime
Ciprofloxacin
Colistin
Gentamicin
Imipenem
Levofloxacin
Meropenem
Norfloxacin
Ofloxacin
Piperacillin
Piperacillin/Tazobactam
Tigecycline
Trimethoprim/Sulfamethoxazole
29


In [17]:
sample_id = out_name
data = [['NA'] * len(new_column_name) for _ in range(len(sample_id))]
df_new = pd.DataFrame(data, columns = new_column_name)
df_new.index =  sample_id

In [18]:
# df_new
for filename in out_name:
    file = pd.read_csv(mdr_folder_path+filename+'.csv')
    # y = sample_id.index(filename)
    y = filename
    # print(y)
    cl = []
    inter  = []
    mic = []
    for x in file.columns:
        if 'Antimicrobial' in x:
            cl.append(x)
        if 'Interpretation' in x:
            inter.append(x)
        if 'MIC' in x:
            mic.append(x)

    for c,i,m  in zip(cl,inter,mic):
        antimicrobial_list = file.loc[:,c]
        value =  file.loc[:,i]
        mic_v =  file.loc[:,m]
        for x in range(len(antimicrobial_list)):
            
            anti_str = clear_str(str(antimicrobial_list[x]))
            # print(anti_str)
            if  anti_str == 'nan':
                continue
            # if not value[x] == '-':
                # print(new_column_dict[anti_str])
            if not str(value[x]) == 'nan':
                df_new.loc[y, new_column_dict[anti_str]]= value[x]
                # print(new_column_dict['MIC_'+anti_str])
            if not str(mic_v[x]) ==  'nan':
                df_new.loc[y, new_column_dict['MIC_'+anti_str]]= mic_v[x]
                    
# df_new.index =  out_name
df_new.to_csv(outputfilename)